In [ ]:
import oBehave as ob
# you will need these libraries for computation & data manipulation
import os
import numpy as np
import pandas as pd
# matplotlib is a standard python visualization package
import matplotlib.pyplot as plt
%matplotlib inline
# seaborn is another library for statistical data visualization
# seaborn style & context settings make plots pretty & legible
import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.markeredgewidth': 2})
sns.set_style('white')
sns.set_palette('deep');
# this makes print statements complaint with python 3 formatting
from __future__ import print_function


# AWS
drive_path = '/data/dynamic-brain-workshop/visual_behavior'
manifest_file = 'visual_behavior_data_manifest.csv' #similar to boc cache file
manifest = pd.read_csv(os.path.join(drive_path,manifest_file))


In [ ]:
#Manifest data only for V1 and choose mouse
VISp_manifest = manifest[(manifest.targeted_structure == "VISp")] #limit dataframe to V1
VISp_exid= VISp_manifest['experiment_id'] #create dataframe of ids

expt=644942849

from visual_behavior.ophys.dataset.visual_behavior_ophys_dataset import VisualBehaviorOphysDataset
dataset = VisualBehaviorOphysDataset(str(expt), cache_dir=drive_path)

trials = dataset.trials

In [ ]:
#From the trial start to the change: How much time passes and how many repeats of the initial image before change?
repeats_per_trial = []
for i in range(len(trials.trial)):
    wait = (trials.change_time[i] - trials.start_time[i])
    repeats = (wait/0.75)
    repeats_per_trial.append([wait, repeats])
    
print(repeats_per_trial[:3])

In [ ]:
#assign engagement binaries only to go trials, skipping catch trials
eng_binary= []
for i in range(len(trials.trial)): #problem if remove ctch trials, differnt length?
    if ((trials.trial_type[i] == 'go') and (trials.response_type[i] == 'HIT')):
        success = 1
    elif((trials.trial_type[i] == 'go') and (trials.response_type[i] == 'MISS')):
        success = 0
    else:
        continue
    eng_binary.append(success)

print(eng_binary)
print(len(eng_binary))


In [ ]:
#code for finding a SIMPLE engagement window (currently the last initial image start to start of response window) times
#go trials only, skipping catch trials

eng_st = []
eng_end = []

for i in range(len(trials.trial)):
    if trials.trial_type[i] == 'go':
        last_initial_image =(trials.change_time[i] - 0.75)
        eng_st.append(last_initial_image)

        resp_window_start = (trials.change_time[i] + 0.15)
        eng_end.append(resp_window_start) 
    else:
        continue
            
print (eng_st[0:4])
print(eng_end[0:4])


In [ ]:
#code for finding a simple engagement window (currently the last initial image start to start of response window) 
#BOTH go and catch trials for more wide binary analysis


eng_st = []
eng_end = []

for i in range(len(trials.trial)):
    last_initial_image =(trials.change_time[i] - 0.75)
    eng_st.append(last_initial_image)

    resp_window_start = (trials.change_time[i] + 0.15)
    eng_end.append(resp_window_start) 
   
            
print (eng_st[0:4])
print(eng_end[0:4])


In [ ]:
#creates list and df column for "success" per trial based on correctly responding to either go or catch
#used for wide engagement binary
trials['success']= 'x'

#create column in dataframe: success
for i in range(len(trials.trial)):
    if ((trials.trial_type[i] == 'go') and (trials.response_type[i] == 'HIT')):
        #create success column
        success = True
    elif ((trials.trial_type[i] == 'catch') and (trials.response_type[i] == 'CR')):
        success= True
    else:
        success = False
        
    trials.success[i]= success #should be bool
 
    

In [ ]:
trials[['trial_type','response_type','success']]

In [ ]:
#wide engagement binary for 2 successes in 3 consecutive trials 
wide_engage_binary = []

for i in range(len(trials.trial)):
    #want to look at window from one trial before to one trial after. unless first of last trial, then there isnt a before or after trial respectively
    if i == 0:
        pre = 0
        post = (i+1)
    elif i== len(trials.trial):
        pre= (i-1)
        post= len(trials.trial)
    else:
        pre= (i-1)
        post= (i+1)
    threetrialwindow = (trials[pre:(post+1)])
    threetrial_successonly = threetrialwindow.success[threetrialwindow.success == True]
    if len(threetrial_successonly) >= 2:
        performing_task = 1
    else:
        performing_task = 0
    wide_engage_binary.append(performing_task)   
    
#wide_engage_binary

#adds wide eng binary as a column in dataset
trials["wide_engage_binary"]= wide_engage_binary

#prints 4 columns to check your work
trials[['trial_type','response_type','success', 'wide_engage_binary']]

In [ ]:
#WideER eng binary for 3 successes out of 5 consecutive trials

wide_engage_binary = []

for i in range(len(trials.trial)):
    #want to look at window from one trial before to one trial after. unless first of last trial, then there isnt a before or after trial respectively
    if i == 0:
        pre = 0
        post = (i+2)
    elif i ==1:
        pre= (i-1)
        post= (i+2)
    elif i== (len(trials.trial)-1):
        pre= (i+2)
        post= (i+1)
    elif i== len(trials.trial):
        pre= (i-2)
        post= len(trials.trial)
    else:
        pre= (i-2)
        post= (i+2)
    threetrialwindow = (trials[pre:(post+1)])
    threetrial_successonly = threetrialwindow.success[threetrialwindow.success == True]
    if len(threetrial_successonly) >= 3:
        performing_task = 1
    else:
        performing_task = 0
    wide_engage_binary.append(performing_task)   
    
#wide_engage_binary

#adds wide eng binary as a column in dataset
trials["wide_engage_binary"]= wide_engage_binary

#prints 4 columns to check your work
trials[['trial_type','response_type','success', 'wide_engage_binary']]

In [ ]:
plt.plot(wide_engage_binary)